In [2]:
# warning 무시
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
# glob : 파일 리스트 뽑기
from glob import glob

# tqdm : for문 진행률 확인
from tqdm import tqdm

from collections import deque

In [4]:
# requests : HTTP를 사용하기 위해 쓰여지는 라이브러리
import requests

# multiprocessing : process spawing
import multiprocessing
import pandas as pd
import numpy as np
import os

# 질문
flag : 수집되는 거 안되는 거 구분? 

recursive = True 용도? 

df['num_lang'] = df['languages'].fillna('').apply(lambda x: len(x.split())) 결측치 공백 입력, x : df['languages'].fillna('') 인가요? 

df.to_csv(f'{steamspy_dir}\\{file_name}.csv', index = False, sep = '\t')
    index = False?

In [5]:
cwd = os.getcwd() # get current working directory : 로컬에서 파일을 가져오려고 cwd -> 같은 위치에 있는 상대경로 활용
print(cwd)
# check_path = glob(f'{cwd}\\**\\2_steampy_tag\steamspy.txt', recursive = True)[0]
# C:\Users\NT550009\Desktop\ASAC_ml_project\web_crawling\JC\2_steampy_tag\steamspy.txt
    

c:\Users\alice\OneDrive\바탕화~1-DESKTOP-K2BRIJ8-1984\ASAC_ml_project\ASAC_ml_project\web_crawling\JC\2_steampy_tag


In [6]:
cwd = os.getcwd() # 로컬에서 파일을 가져오려고 cwd -> 같은 위치에 있는 상대경로 활용
print(cwd)
check_path = glob(f'{cwd}\\**\\steamspy.txt', recursive = True)[0]
print(check_path)
# C:\Users\NT550009\Desktop\ASAC_ml_project\web_crawling\JC\2_steampy_tag\steamspy.txt

c:\Users\alice\OneDrive\바탕화~1-DESKTOP-K2BRIJ8-1984\ASAC_ml_project\ASAC_ml_project\web_crawling\JC\2_steampy_tag
c:\Users\alice\OneDrive\바탕화~1-DESKTOP-K2BRIJ8-1984\ASAC_ml_project\ASAC_ml_project\web_crawling\JC\2_steampy_tag\steamspy.txt


In [7]:
def extract(appid):
    flag = ''
    cwd = os.getcwd() # 로컬에서 파일을 가져오려고 cwd -> 같은 위치에 있는 상대경로 활용
    check_path = glob(f'{cwd}\\**\\steamspy.txt', recursive = True)[0] # \\**\\ 경로 설정을 편하게. 성현님 코드에서 txt와 ipynb 가 다른 곳에 있음.
   
    # steamspy.txt에서 모을 내용 리스트 원소로 정리
    check_list = [i.replace('\n','') for i in open(check_path).readlines()]
    
    # appid 당 check_list 원소 정리
    result_dict = {appid : {i : '' for i in check_list}}
    # sample : https://steamspy.com/api.php?request=appdetails&appid=1794680
    res = requests.get(f'https://steamspy.com/api.php?request=appdetails&appid={appid}')
    
    # 요청이 성공적으로 수행되지 않았을 때
    if res.status_code != 200: # status_code : 응답상태 확인
        flag = False
        return result_dict, flag
    
    # json 형식 dict으로 가져오기
    res_json = res.json() 
    
    for check in check_list:
        # 예외 처리
        # res_json 에서 check 키에 내용 찾아서 추가
        try:
            result_dict[appid][check] = res_json[check]
        except:
            continue
        
    flag = True
    
    return result_dict, flag
        
        
    

In [8]:
def run():
    cwd = os.getcwd() 
    pwd = os.path.abspath(os.path.join(cwd, '..'))

    appid_dirs = glob(f'{pwd}\\**\\1_appid\\*.txt',recursive=True)
    check_path = glob(f'{cwd}\\**\\steamspy.txt', recursive = True)[0]
    
    check_list = [i.replace('\n','') for i in open(check_path).readlines()]
    
    for appid_dir in appid_dirs:
        base_name = os.path.basename(appid_dir) 
        if '2019' in base_name or '2018' in base_name: # 2018, 2019 pass
            continue
        app_list = [i.replace('\n','') for i in open(appid_dir).readlines()] 
        
        
        # 딕셔너리 컴프리헨션
        result_last= {i : deque([]) for i in check_list} # {'developer': deque([]), 'publisher': deque([]) } 형식
        result_last["appid"] = app_list
        
        result_mid = {}
        result_error = {}
        
        with multiprocessing.Pool() as p: # 병렬처리
            for val, flag in tqdm(p.imap_unordered(extract, app_list, 16), total = len(app_list)):
                result_mid.update(val)
                if flag == False:
                    result_error.update(val)
                    
        for app in app_list:
            for check in check_list:
                result_last[check].append(result_mid[app][check])
                
        df = pd.DataFrame(result_last)
        
        # 결측치 처리
        df['price'] = df['price'].fillna(0)
        df['price'] = df['price'].astype(int) / 100
        
        df['initialprice'] = df['initialprice'].fillna(0)
        df['initialprice'] = df['initialprice'].astype(int) / 100
        
        df['num_lang'] = df['languages'].fillna('').apply(lambda x: len(x.split())) 
        
        file_name = os.path.splitext(base_name)[0]
        
        df.to_csv(f'{steamspy_dir}\\{file_name}.csv', index = False, sep = '\t')
        
if __name__ == '__main__':
    run()
                    
        
        
    

  0%|          | 0/250 [00:00<?, ?it/s]

In [ ]:
    for appid_dir in appid_dirs:
        base_name = os.path.basename(appid_dir) 

In [ ]:
check_path = glob(f'{cwd}\\**\\steamspy.txt', recursive = True)[0]
check_list = [i.replace('\n','') for i in open(check_path).readlines()]
result_last= {i : deque([]) for i in check_list} 
print(result_last)

{'developer': deque([]), 'publisher': deque([]), 'positive': deque([]), 'negative': deque([]), 'average_forever': deque([]), 'average_2weeks': deque([]), 'median_forever': deque([]), 'median_2weeks': deque([]), 'price': deque([]), 'initialprice': deque([]), 'languages': deque([]), 'tags': deque([]), 'genre': deque([])}
